In [4]:
import torch
from torch import nn
"""
当添加通道时,输入和隐藏的表示都变成了三维张量。
例如,每个RGB输入图像具有3*h*w的形状。
将这个大小为3的轴称为通道维度
"""

# 一、多输入通道
"""
当输入包含多个通道时,需要构造一个与输入数据具有相同输入通道数的卷积核,以便与输入数据进行互相关运算。
假设输入的通道数为c,那么卷积核的输入通道数也需要为c。
如果卷积核的窗口形状是kh*kw,当c = 1时,可以把卷积核看作形状为 kh*kw 的二维张量

当c > 1时,我们卷积核的每个输入通道将包含形状为 kh*kw 的张量。将这些张量 c[i] 连结在一起可以
得到形状为 c[i]*kh*kw 的卷积核。由于输入和卷积核都有c个通道,可以对每个通道输入的二维张量和
卷积核的二维张量进行互相关运算,再对通道求和(将c[i]的结果相加)得到二维张量。
这是多通道输入和多输入通道卷积核之间进行二维互相关运算的结果。
"""
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros(X.shape[0]-h+1, X.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h, j:j+w] * K).sum()
    return Y
def corr2d_multi_in(X, K): # 相当于一个输入通道对应一个卷积核,最后将每个通道的卷积结果相加
    return sum(corr2d(x, k) for x, k in zip(X, K))
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], 
                  [[1.0, 2.0], [3.0, 4.0]]])
print(f"多通道输入的卷积:\n{corr2d_multi_in(X,K)}")

# 二、多输出通道
"""
在最流行的神经网络架构中，随着神经网络层数的加深，常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度。
直观地说，可以将每个通道看作对不同特征的响应。而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。
因此，多输出通道并不仅是学习多个单通道的检测器。

用ci和co分别表示输入和输出通道的数目,并让kh和kw为卷积核的高度和宽度。为了获得多个通道的输出,
可以为每个输出通道创建一个形状为 ci*kh*kw 的卷积核张量,这样卷积核的形状是 co*ci*kh*kw。
在互相关运算中，每个输出通道先获取所有输入通道，再以对应该输出通道的卷积核计算出结果。
"""
def corr2d_multi_in_out(X, K): # 计算多个通道的输出的互相关函数
    # 迭代K的第0个维度，每次都对输入X执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], dim=0)
# 通过将核张量K与K+1(K中每个元素加1)和K+2连接起来，构造了一个具有3个输出通道的卷积核。
K = torch.stack((K,K+1,K+2), dim=0)
print(f"K.shape:{K.shape}")
print(f"多通道输出的卷积:\n{corr2d_multi_in_out(X,K)}") # 输出包含三个通道,第一个通道与X和K的多输入通道卷积一致

# 三、1*1卷积层
"""
1*1卷积层没有提取相邻像素特征关系的能力,
可将其看作一个在每个像素位置应用的全连接层,实现从ci个输入值到co个输出值的转换,
且权重一致(权重维度 co*ci + 偏置b), 指的是对于每一维的输入,进行变换的权重是一致的,不同维度的权重是不一样的
1*1卷积层通常用于调整网络层的通道数量和控制模型复杂性
"""
def corr2d_multi_in_out_1x1(X, K): # 注意需要对输入和输出的数据形状进行调整
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w)) # 每个通道,二维像素矩阵到一维向量
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))
# 1*1的卷积运算相当于实现数corr2d_multi_in_out
X = torch.normal(mean=0, std=1, size=(3, 3, 3))
K = torch.normal(mean=0, std=1, size=(2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
print(float(torch.abs(Y1 - Y2).sum()) < 1e-6)


多通道输入的卷积:
tensor([[ 56.,  72.],
        [104., 120.]])
K.shape:torch.Size([3, 2, 2, 2])
多通道输出的卷积:
tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])
True
